In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [31]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Mestrado/pre_processing_important.csv')
df['labels'].fillna('', inplace=True)

In [3]:
mappings = {
    '0': 'information_support',
    '1': 'information_support',
    '2': 'information_support',

    '3':  'emotional_support',
    '4': 'emotional_support',
    '5': 'emotional_support',
    '6': 'emotional_support',
    '7': 'emotional_support',

    '8' : 'esteem_support',
    '9': 'esteem_support',
    
    '10': 'network_support',
    '11': 'network_support',

    #'12': 'perform_direct_task',
    #'13': 'express_willingness',

    '14': 'seeking_support',
    '15': 'seeking_support',

    '16': 'gratitude',
    '17': 'congratulations',
    '18' :'sharing_personal_experiences' 

}


mappings_first = {
    '0': 'offering_support',
    '1': 'offering_support',
    '2': 'offering_support',

    '3':  'offering_support',
    '4': 'offering_support',
    '5': 'offering_support',
    '6': 'offering_support',
    '7': 'offering_support',

    '8' : 'offering_support',
    '9': 'offering_support',
    
    '10': 'offering_support',
    '11': 'offering_support',

    '12': 'offering_support',
    '13': 'offering_support',

    '14': 'seeking_support',
    '15': 'seeking_support',

    '16': 'group_interactions',
    '17': 'group_interactions',
    '18' : 'group_interactions' 

}

def do(labels):
  labels = labels.split(',')
  new_labels = []
  for label in labels:
    if label in mappings_first:
      new_labels.append(mappings_first[label])
  return new_labels
df['labels'] = df['labels'].apply(do)
df = df[df['labels'].apply(len) > 0]
### this is required due to the stratified algorithm split used
df = df.reset_index(drop=True)

In [4]:
!pip install tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import tensorflow_hub as hub
import numpy as np
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

from nltk.tokenize import sent_tokenize



In [6]:
np.array([0.] * 512).shape

(512,)

In [34]:
import nltk
nltk.download('punkt')


def vectorize_sentences(X, number):

    new_X = []
    for instance in X:
      sentences = sent_tokenize(instance)
      current_embedding = embed(sentences[:number]).numpy()
      
      diff = number -len(current_embedding)
      if diff > 0:
          padding = np.zeros((diff, 512))
          current_embedding = np.concatenate((padding, current_embedding), axis=0)
      new_X.append(current_embedding)
    
    return np.array(new_X)


ans = vectorize_sentences(df['text'], 7)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
ans.shape

(453, 7, 512)

In [37]:
from ast import literal_eval
df['labels'] = df['labels'].apply(literal_eval)

In [38]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y=mlb.fit_transform(df['labels'])

In [39]:
y.shape

(453, 3)

In [12]:
#### we can transform the input here #####

import tensorflow as tf
from sklearn.decomposition import PCA

class PCA_Layer(tf.keras.layers.Layer):
    """
    This layer computes the PCA transform of the data.
    """
    def __init__(self, means, principal_axes):
        super(PCA_Layer, self).__init__()
        self.means = means
        self.principal_axes = tf.constant(principal_axes)
        
    def call(self, inputs):
        return tf.matmul(inputs-self.means, tf.transpose(tf.cast(self.principal_axes, inputs.dtype)))
    
class PCA_trainer():
    def __init__(self, output_dims):
        self.obj = PCA(output_dims)
        self.model = None
        
    def fit(self, embedded_sentences):
        self.obj.fit(embedded_sentences)
        
    def stats(self):
        print("Cumulative explained variance:")
        print(self.obj.explained_variance_ratio_.cumsum())
        
    def build_tf_model(self, input_dim = 512):
        # Extract the info from the sklearn model
        pca_layer = PCA_Layer(self.obj.mean_, self.obj.components_)
        
        input_ = tf.keras.Input(shape = (input_dim), dtype = tf.float32)
        output_ = pca_layer(input_)
        self.model = tf.keras.Model(input_, output_)
    
    def save(self, output_path):
        self.model.save(output_path)

In [13]:
df_unknown = pd.read_csv('/content/gdrive/MyDrive/Mestrado/semi_supervised_learning.csv')

In [14]:
df_unknown.shape

(29283, 6)

In [107]:
import nltk
nltk.download('punkt')

sentences = []
for sentences_instances in df_unknown['text'].apply(lambda x : np.array(sent_tokenize(x))).tolist():
  sentences += sentences_instances.tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
len(sentences)

189253

In [108]:
from sklearn.decomposition import PCA
import tensorflow as tf
import tensorflow_hub as hub 

embedded_sentences = [embed([i]) for i in sentences]
embedded_sentences = np.array(embedded_sentences).reshape(len(sentences), 512)

In [18]:
#embedded_sentences.shape

In [19]:
#obj = PCA_trainer(128)
#obj.fit(embedded_sentences)
#obj.build_tf_model()
#encoder = obj.model

In [20]:
#encoded_samples = []
#for ele in ans:
#  encoded_samples.append(encoder.predict(ele))

In [27]:
ans

array([], dtype=float64)

In [21]:
#encoded_samples = np.array(encoded_samples)
#ncoded_samples.shape

In [40]:
from sklearn.model_selection import train_test_split
# encoded_samples
X_train, X_test, y_train, y_test = train_test_split(ans, y, test_size=0.2, random_state=42)


In [114]:
new_train = []
for x in X_train:
  new_train.append(nn.encoder.predict(x))
new_train = np.array(new_train)

In [115]:
new_test = []
for x in X_test:
  new_test.append(nn.encoder.predict(x))
new_test = np.array(new_test)

https://dl.acm.org/doi/pdf/10.1145/3232116.3232152?casa_token=7WeZ536Eyl4AAAAA:zV7TxZqL8vZkgoP39-EoBZv-W5OnzZkQ1W853TWPzAqEfbbG63byEekeeSnXmBdBQKB8oNeI_sw

In [130]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM, GRU, Input,  Bidirectional, Flatten
from keras.layers import Dense, Dropout, LeakyReLU, Conv1D, MaxPool1D, GlobalMaxPool1D
from keras.regularizers import l1


# define model
model = Sequential()
#model.add(Conv1D(filters=4, kernel_size=3,strides=1, padding="causal", activation="relu", input_shape=(7, 128), activity_regularizer=l1(0.0001) ) )
#model.add(Conv1D(filters=8, kernel_size=3,strides=1, padding="causal", activation="relu", activity_regularizer=l1(0.0001)) )
#model.add(Conv1D(filters=8, kernel_size=3,strides=1, padding="causal", activation="relu") )
#model.add(MaxPool1D())
#model.add(Flatten())
model.add(Input(shape=(7, 128)))
model.add(Bidirectional(LSTM(5, activation=LeakyReLU(alpha=0.1), activity_regularizer=l1(0.0001))))
model.add(Dense(48))
model.add(Dropout(0.3))
model.add(Dense(3))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["categorical_accuracy"])

In [131]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_6 (Bidirectio  (None, 10)               5360      
 nal)                                                            
                                                                 
 dense_38 (Dense)            (None, 48)                528       
                                                                 
 dropout_17 (Dropout)        (None, 48)                0         
                                                                 
 dense_39 (Dense)            (None, 3)                 147       
                                                                 
Total params: 6,035
Trainable params: 6,035
Non-trainable params: 0
_________________________________________________________________


In [47]:
X_train.shape, X_test.shape, y_train.shape

((362, 7, 512), (91, 7, 512), (362, 3))

In [133]:
model.fit(new_train, y_train, epochs=25, batch_size=8, validation_data=(new_test, y_test))

Epoch 1/25
46/46 [==============================] - 0s 10ms/step - loss: 0.6294 - categorical_accuracy: 0.5138 - val_loss: 0.6421 - val_categorical_accuracy: 0.3846
Epoch 2/25
46/46 [==============================] - 0s 10ms/step - loss: 0.6548 - categorical_accuracy: 0.4530 - val_loss: 0.6568 - val_categorical_accuracy: 0.3297
Epoch 3/25
46/46 [==============================] - 0s 9ms/step - loss: 0.6229 - categorical_accuracy: 0.5470 - val_loss: 0.6196 - val_categorical_accuracy: 0.5824
Epoch 4/25
46/46 [==============================] - 0s 9ms/step - loss: 0.6020 - categorical_accuracy: 0.5580 - val_loss: 0.7306 - val_categorical_accuracy: 0.3626
Epoch 5/25
46/46 [==============================] - 0s 9ms/step - loss: 0.6173 - categorical_accuracy: 0.5525 - val_loss: 0.6539 - val_categorical_accuracy: 0.5824
Epoch 6/25
46/46 [==============================] - 0s 10ms/step - loss: 0.6093 - categorical_accuracy: 0.5884 - val_loss: 0.7492 - val_categorical_accuracy: 0.5934
Epoch 7/25
46

In [134]:
y_pred = model.predict(new_test)

In [135]:
thresholds = np.arange(0, 1, 0.005)

In [136]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix, f1_score, accuracy_score # we can use gmean

def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')
 

scores = [f1_score(y_test, to_labels(y_pred, t), average="weighted") + accuracy_score(y_test, to_labels(y_pred, t)) for t in thresholds]

In [137]:
ix = np.argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.390, F-Score=0.93222


In [138]:
from sklearn.metrics import jaccard_score, hamming_loss
y_pred = y_pred > thresholds[ix]
print(classification_report(y_test, y_pred))
print(multilabel_confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(hamming_loss(y_test, y_pred))
print(jaccard_score(y_test, y_pred, average="weighted"))

              precision    recall  f1-score   support

           0       0.58      0.89      0.70        46
           1       0.52      1.00      0.69        46
           2       1.00      0.18      0.30        28

   micro avg       0.56      0.77      0.65       120
   macro avg       0.70      0.69      0.56       120
weighted avg       0.66      0.77      0.60       120
 samples avg       0.59      0.75      0.63       120

[[[15 30]
  [ 5 41]]

 [[ 3 42]
  [ 0 46]]

 [[63  0]
  [23  5]]]
0.32967032967032966
0.3663003663003663
0.4488437001594896


In [102]:
import nltk
nltk.download('punkt')

sentences = []
for sentences_instances in df['text'].apply(lambda x : np.array(sent_tokenize(x))).tolist():
  sentences += sentences_instances.tolist()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [106]:
len(sentences)

2792

In [104]:
embedded_sentences = [embed([i]) for i in sentences]


In [105]:
len(embedded_sentences[0])

1

In [ ]:
from sklearn.decomposition import PCA
import tensorflow as tf
import tensorflow_hub as hub 


embedded_sentences = [embed([i]) for i in sentences]

embedded_sentences = np.array(embedded_sentences).reshape(1055, 512)

In [ ]:
len(embedded_sentences), len(embedded_sentences[0])

In [109]:
class DenseTranspose(tf.keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = tf.keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",    initializer="zeros",shape=[self.dense.input_shape[-1]])
        super().build(batch_input_shape)
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
        return self.activation(z + self.biases)


def get_encoder(autoencoder, n_bottleneck):
    idx = np.where([layer.output.shape[1]==n_bottleneck for layer in autoencoder.layers])[0][-1]
    bottleneck = autoencoder.layers[idx].output
    encoder = tf.keras.Model(inputs=autoencoder.input, outputs=bottleneck)
    return encoder

class TIED_WEIGHTS_NORMALIZED_AUTOENCODER_trainer():
    def __init__(self, input_dims, middle_dims, bottleneck_dims):
        self.input_dims = input_dims
        self.middle_dims = middle_dims
        self.bottleneck_dims = bottleneck_dims
        self.autoencoder = self.build_autoencoder()
        self.encoder = None
        self.trained = False
        
    def build_autoencoder(self):
        
        dense_1 = tf.keras.layers.Dense(self.middle_dims, activation="selu")
        dense_2 = tf.keras.layers.Dense(self.bottleneck_dims, activation="selu")
        
        #tied_encoder
        inputs = tf.keras.Input(shape=(self.input_dims))
        l_flatten = tf.keras.layers.Flatten()(inputs)
        l_en1 = dense_1(l_flatten)
        l_en2 = dense_2(l_en1)

        # normalized bottleneck
        l_en2 = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis = 1))(l_en2)

        #tied_decoder
        l_dc1 = DenseTranspose(dense_2, activation="selu")(l_en2)
        l_dc2 = DenseTranspose(dense_1, activation="linear")(l_dc1)
        
        return tf.keras.models.Model(inputs, l_dc2)
    
    def train(self, embedded_sentences,  bs = 128, lr = 1e-4, ep = 100, verbose = 0):
        
        self.autoencoder.compile(loss="mse",optimizer=tf.keras.optimizers.Adam(learning_rate=lr))
        print(self.autoencoder.summary())

        self.autoencoder.fit(embedded_sentences, 
                    embedded_sentences, 
                    epochs=ep, 
                    batch_size=bs, 
                    verbose=verbose, 
                    callbacks = [])
        
        self.trained = True
        
    def build_tf_model(self):
        if not self.trained:
            print("Model is not trained, encoder has not been built.")
        else:
            encoder = get_encoder(self.autoencoder, self.bottleneck_dims)
            output_ = encoder(encoder.input)
            self.encoder = tf.keras.Model(encoder.input, output_)
        
    def save(self, output_path):
        self.encoder.save('./{}/encoder'.format(output_path))
        self.autoencoder.save('./{}/autoencoder'.format(output_path))

In [110]:
nn  = TIED_WEIGHTS_NORMALIZED_AUTOENCODER_trainer(512, 256, 128)
nn.build_autoencoder()
nn.train(embedded_sentences)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 512)]             0         
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_30 (Dense)            (None, 256)               131328    
                                                                 
 dense_31 (Dense)            (None, 128)               32896     
                                                                 
 lambda (Lambda)             (None, 128)               0         
                                                                 
 dense_transpose (DenseTrans  (None, 256)              33152     
 pose)                                                           
                                                             

In [111]:
nn.build_tf_model()

In [112]:
embedded_sentences[0].shape

(512,)

In [113]:
nn.encoder.predict(embedded_sentences).shape

(189253, 128)

In [ ]:
!git clone https://github.com/sdilbaz/MedHelp-Data-Collection.git

In [ ]:
!python ./MedHelp-Data-Collection/scrape-medhelp.py / --ncpu 3

In [140]:
X_train.shape

(362, 7, 512)

In [ ]:
import numpy as np
import cPickle as pickle
from sklearn.decomposition import PCA
import subprocess

Glove = {}
f = open('/home/vikas/Desktop/glove.6B/glove.6B.300d.txt')


print("Done.")
X_train = []
X_train_names = []
for x in Glove:
        X_train.append(Glove[x])
        X_train_names.append(x)

X_train = np.asarray(X_train)
pca_embeddings = {}

# PCA to get Top Components
pca =  PCA(n_components = 300)
X_train = X_train - np.mean(X_train)
X_fit = pca.fit_transform(X_train)
U1 = pca.components_

z = []

# Removing Projections on Top Components
for i, x in enumerate(X_train):
	for u in U1[0:7]:        
        	x = x - np.dot(u.transpose(),x) * u 
	z.append(x)

z = np.asarray(z)

# PCA Dim Reduction
pca =  PCA(n_components = 150)
X_train = z - np.mean(z)
X_new_final = pca.fit_transform(X_train)


# PCA to do Post-Processing Again
pca =  PCA(n_components = 150)
X_new = X_new_final - np.mean(X_new_final)
X_new = pca.fit_transform(X_new)
Ufit = pca.components_

X_new_final = X_new_final - np.mean(X_new_final)

final_pca_embeddings = {}
embedding_file = open('pca_embed2.txt', 'w')

for i, x in enumerate(X_train_names):
        final_pca_embeddings[x] = X_new_final[i]
        embedding_file.write("%s\t" % x)
    	for u in Ufit[0:7]:
            final_pca_embeddings[x] = final_pca_embeddings[x] - np.dot(u.transpose(),final_pca_embeddings[x]) * u 

        for t in final_pca_embeddings[x]:
                embedding_file.write("%f\t" % t)
        
        embedding_file.write("\n")


print("Results for the Embedding")
print subprocess.check_output(["python", "all_wordsim.py", "pca_embed2.txt", "data/word-sim/"])
print("Results for Glove")
print subprocess.check_output(["python", "all_wordsim.py", "../glove.6B/glove.6B.300d.txt", "data/word-sim/"])